In [25]:
from kitti360scripts.helpers.ply import read_ply
from kitti360scripts.helpers.project import CameraPerspective, CameraFisheye
from kitti360scripts.helpers.labels import id2label
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import open3d
import skimage.segmentation
from scipy.ndimage import generic_filter

In [2]:
ball = np.load('data_3d_semantics/2013_05_28_drive_0000_sync/ball/0000000009.npy')
u, v, mask = [], [], []
for i in range(6):
    u_i = np.load(f'data_3d_projection/2013_05_28_drive_0000_sync/0000000009/u_{i}.npy')
    v_i = np.load(f'data_3d_projection/2013_05_28_drive_0000_sync/0000000009/v_{i}.npy')
    mask_i = np.load(f'data_3d_projection/2013_05_28_drive_0000_sync/0000000009/mask_{i}.npy')
    u.append(u_i)
    v.append(v_i)
    mask.append(mask_i)

In [3]:
ply = read_ply('KITTI_360/data_3d_semantics/train/2013_05_28_drive_0000_sync/static/0000000002_0000000385.ply')
points = np.array([ply['x'], ply['y'], ply['z']]).T
colors = np.array([ply['red'], ply['green'], ply['blue']]).T

In [4]:
points = points[ball]
colors = colors[ball]

In [5]:
# find 30 nearest neighbors for each point
k = 30
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(points)
pcd.colors = open3d.utility.Vector3dVector(colors/255)
pcd_tree = open3d.geometry.KDTreeFlann(pcd)
nn = np.apply_along_axis(lambda x: pcd_tree.search_knn_vector_3d(x, k)[1], 1, points)

In [6]:
nn.shape, points.shape

((2082527, 30), (2082527, 3))

In [7]:
pcd.estimate_normals()
# pcd.orient_normals_to_align_with_direction()

In [8]:
normals = np.asarray(pcd.normals)

In [9]:
graph = points[nn]

In [10]:
graph.shape

(2082527, 30, 3)

In [11]:
d = np.linalg.norm(graph - points[:, None], axis=2)

In [12]:
d[0]

array([0.        , 0.21951345, 0.25984392, 0.28300527, 0.28603223,
       0.30613613, 0.3163502 , 0.34459704, 0.35724762, 0.37576213,
       0.38553807, 0.4181843 , 0.43098176, 0.44537148, 0.4630081 ,
       0.49300683, 0.5067244 , 0.5140843 , 0.53338224, 0.5602433 ,
       0.581545  , 0.6036026 , 0.621105  , 0.62149227, 0.6282998 ,
       0.629953  , 0.65664446, 0.67465156, 0.6773149 , 0.6812838 ],
      dtype=float32)

In [13]:
cosine = np.sum(normals[:, None] * normals[nn], axis=2) / (np.linalg.norm(normals[:, None], axis=2) * np.linalg.norm(normals[nn], axis=2))
cosine = cosine.clip(-1, 1)
angle = np.arccos(cosine)

In [14]:
angle.max(), angle.min()

(3.1415888043859335, 0.0)

In [15]:

sine0 = np.linalg.norm(np.cross(np.repeat(normals[0:1], k, axis=0), normals[nn[0]]), axis=1) / (np.linalg.norm(normals[0]) * np.linalg.norm(normals[nn[0]], axis=1))

In [16]:
sine0.shape

(30,)

In [17]:
cosine[0].shape

(30,)

In [18]:
angle0 = np.arctan2(sine0, cosine[0])

In [19]:
angle0

array([0.        , 0.        , 0.01532337, 0.01858996, 0.        ,
       0.01532337, 0.01532337, 0.        , 3.12072127, 0.01532337,
       3.09386409, 0.03512225, 0.03306361, 0.03454636, 0.05589274,
       0.05900688, 0.03315942, 0.03585439, 0.0544148 , 0.05746518,
       0.05725234, 0.08227554, 0.03585439, 3.09176034, 3.09902265,
       3.06035234, 3.06703197, 3.05856673, 3.07282282, 0.05098715])

In [20]:
angle[0]

array([0.        , 0.        , 0.01532337, 0.01858996, 0.        ,
       0.01532337, 0.01532337, 0.        , 3.12072127, 0.01532337,
       3.09386409, 0.03512225, 0.03306361, 0.03454636, 0.05589274,
       0.05900688, 0.03315942, 0.03585439, 0.0544148 , 0.05746518,
       0.05725234, 0.08227554, 0.03585439, 3.09176034, 3.09902265,
       3.06035234, 3.06703197, 3.05856673, 3.07282282, 0.05098715])

In [21]:
# check all close
np.allclose(angle0, angle[0])

True

In [22]:
angle.min()

0.0

In [23]:
# v is number of points in which both points are visible
v = np.zeros((len(points), k))
for i in range(6):
    mask_i = mask[i]
    v += mask_i[nn] * mask_i[:, None]

In [24]:
v.shape

(2082527, 30)

In [47]:
L = 6

In [ ]:
o = np.zeros((len(points), k))
for i in range(6):
    mask_i = mask[i]
    o += mask_i[nn] * mask_i[:, None] * angle

In [81]:
import numpy as np
from scipy.ndimage import generic_filter

def create_feature_tensor(image, t=5):
    S = image.shape[0]
    def count_k(local_area, k):
        return np.sum(local_area == k)
    
    N = image.max() + 1
    result = np.zeros((S, S, N), dtype=np.int32)

    for k in np.unique(image):
        result[:, :, k] = generic_filter(image, count_k, footprint=np.ones((t, t)), mode='constant', cval=np.nan, extra_arguments=(k,))
        
    return result

In [82]:

# Example
S, N = 10, 4  # Define size of grid and range of values
np.random.seed(0)  # For reproducibility
matrix = np.random.randint(0, N, size=(S, S))  # Random matrix
feature_tensor = create_feature_tensor(matrix)
print(feature_tensor.shape)  # Should be (S, S, N)


(10, 10, 4)


In [83]:
matrix

array([[0, 3, 1, 0, 3, 3, 3, 3, 1, 3],
       [1, 2, 0, 3, 2, 0, 0, 0, 2, 1],
       [2, 3, 3, 2, 0, 1, 1, 1, 1, 0],
       [1, 0, 3, 0, 3, 1, 2, 3, 3, 0],
       [2, 3, 0, 1, 3, 1, 3, 3, 2, 3],
       [0, 1, 1, 1, 3, 0, 3, 2, 0, 3],
       [3, 2, 3, 2, 3, 0, 2, 0, 0, 0],
       [1, 1, 2, 0, 0, 1, 3, 0, 1, 2],
       [2, 3, 0, 1, 1, 3, 1, 1, 3, 2],
       [3, 3, 2, 2, 3, 0, 2, 3, 1, 0]])

In [84]:
feature_tensor[2,2]

array([7, 4, 5, 9], dtype=int32)